<a href="https://colab.research.google.com/github/YKannikanti/Yasasri-/blob/main/midterm/midterm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Mid-term for HDS5210

Your supervisor is concerned about 4-year survival risks for COPD. She has asked for you to do some analysis using a new metric, BODE. BODE is an improvement on a previous metric and promises to provide insight on survival risks.

BODE is defined here. https://www.mdcalc.com/calc/3916/bode-index-copd-survival#evidence

Your assignment is to create a BODE calculation, use it to calculate BODE scores and BODE survival rates for a group of patients. Then we want to evaluate the average BODE scores and BODE survival rates for each area hospital.

Your patient input file will have the following columns:
NAME,SSN,LANGUAGE,JOB,HEIGHT_M,WEIGHT_KG,fev_pct,dyspnea_description,distance_in_meters,hospital

BODE calculations require a BMI value, so you will have to create a function for it.

Your output should be in the form of two CSV files, patient_output.csv and hospital_output.csv.

Patient_output will have the following columns:
NAME,BODE_SCORE,BODE_RISK,HOSPITAL

Hospital output will have the following columns:
HOSPITAL_NAME, COPD_COUNT, PCT_OF_COPD_CASES_OVER_BEDS, AVG_SCORE, AVG_RISK

Each function you create should have documentation and a suitable number of test cases. If the input data could be wrong, make sure to raise a Value Error.

For this assignment, use the doctest, json, and csv libraries. Pandas is not allowed for this assignment.

In [108]:
import doctest
import csv
import json

### Step 1: Calculate BMI

In [109]:
import doctest

def calculate_bmi(height_m, weight_kg):
  """
  calculate body mass index

  parameters:
    weight_kg (float):weight in kilograms
    height_m (float) :heiht in meters






  >>> calculate_bmi(1.80, 70)
  22.857142857142858"""

  if not (0 < height_m < 3):
    raise ValueError
  if not (0 < weight_kg < 500):
    raise ValueError

  return weight_kg / (height_m ** 2)

### Step 2: Calculate BODE Score

In [110]:
def calculate_bode_score(fev_pct, bmi):
  """
  if not (0 <= fev_pct <= 100):
    raise ValueError
  if dyspnea_description == "dyspnea":
    raise ValueError
  if distance_in_meters < 0:
    raise ValueError
  """
  #fev scoring
  if fev_pct >= 95:
    fev_score = 4
  elif fev_pct >= 90:
    fev_score = 3
  elif fev_pct >= 80:
    fev_score = 2
  else:
    fev_score = 1

  #scoring for dyspnea
  dyspnea_score= dyspnea_description

  #scoring for distance
  if distance_in_meters < 100:
    distance_score =0
  elif distance_in_meters < 200:
    distance_score = 1
  elif distance_in_meters < 300:
    distance_score = 2
  else:
    distance_score = 3

    #total bode score
  return fev_score + distance_score + dyspnea_score




### Step 3: Calculate BODE Risk

In [111]:
def calculate_bode_risk(bode_score):

  """calculate the bode risk"""
  if bode_score <= 2:
    return "low risk"
  elif bode_score <= 4:
    return "medium risk"
  elif bode_score <= 6:
    return "high risk"
  else :
    return "very high risk"



### Step 4: Load Hospital Data

In [112]:
def load_hospital_data(hospital_json):
  with open(hospital_json) as f:
    return json.load(f)

### Step 5: Main business logic

Call BODE Score, BODE Risk functions for each patient.

For each hospital, calculate Avg BODE score and Avg BODE risk and count the number of cases for each hospital.

In [113]:
def main(patient_csv, hospital_json, patient_output_file, hospital_output_file):
  patient_results = []
  hospital_output_list = []

 #load hospital data
  hospital_data=load_hospital_data(hospital_json)

  with open(patient_csv, 'r') as f:
    reader = csv.DictReader(f)
    for row in reader:

      name  = row['sriya']
      height_m = float(row['height_m'])
      weight_kg = float(row['weight_kg'])
      fev_pct = float(row['fev_pct'])
      dyspnea_description = row['dyspnea_description']
      distance_in_meters = float(row['distance_in_meters'])
      hospital = row['hospital']

      #calculate bmi
      bmi = calculate_bmi(height_m, weight_kg)

      #calculate bode score
      bode_score = calculate_bode_score(fev_pct, bmi)

      #calculate bode risk
      bode_risk = calculate_bode_risk(bode_score)


      append_patient_results = [{
        'NAME':name,
        'BODE_SCORE': bode_score,
        'BODE_RISK': bode_risk,
        'HOSPITAL': hospital
    }]
    patient_results.extend(append_patient_results)
    #initialize hospital metrics if not already present

    if hospital_name not in hospital_metrics:
      hospital_metrics[hospital_name] = {'beds': 0, 'copd_count': 0, 'avg_risk': 0, 'avg_score': 0}

    # update hospital statistics

    for hospital, stats in hospital_data.items():
            stats['copd_count'] =  stats.get('copd_count',0) +1
            stats['avg_risk'] = stats.get =('avg_risk',0) + bode_risk
            stats['avg_score'] = stats.get('avg_score',0) + (1 if bode_score == 'low risk'else 0)

    hospital_output_list=[]

  #calculate hospital metrics
    for hospital, stats in hospital_data.items():
      copd_count = stats['copd_count']
      avg_score = stats['avg_score'] / copd_count
      avg_risk = stats['avg_risk'] / copd_count

    #Wrte Patient_output.csv
    with open(patient_output_file, 'w', newline='') as f:
        writer = csv.writer(f)
        for result in patient_results:
         writer.writeheader()
         writer.writerow(result)


  #Write Hospital_output.csv
  with open(hospital_output_file, 'w', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=['hospital_name', 'copd_count', 'pct_of_copd_cases_over_beds', 'avg_score', 'avg_risk'])
        writer.writeheader()
        for hospital, stats in hospital_data.items():
            stats['avg_score'] /= stats['copd_count']
            stats['avg_risk'] /= stats['copd_count']
            stats['pct_of_copd_cases_over_beds'] = stats['copd_count'] / stats['beds'] * 100
            writer.writerow({
                 'hospital_name': hospital,
                 'copd_count': stats['copd_count'],
                 'pct_of_copd_cases_over_beds': stats['pct_of_copd_cases_over_beds'],
                 'avg_score': stats['avg_score'],
                 'avg_risk': stats['avg_risk']
             })

